# LIBRARIES

In [2]:
import Learner
from Environment import Environment
from UserCat import UserCat
from Product import Product
import numpy as np
import random

# TESTING of ENVIRONMENT.py

In [3]:
users=[]
products=[]

nameofproduct= [ #name of products
    "Calabazas",
    "Hinojo",
    "Sesamo",
    "Girasol",
    "Amapola"
]

prices=[[4., 5, 6 ,7],
    [9., 10, 11, 12],
    [18., 19, 20, 21],
    [24., 25, 26, 27],
    [33., 34, 35, 36]]
#1-2 di delta, Con sovrapposizione

cost=[2,4.5,9,14,17]

#sarebbe interessante anche prendere da file il tutto così da cambiare tutto più facilmente
#calcolo i margini dai cost mi sembra più sensato e anche più veloce se dobbiamo cambiare continuamente

cost2 = np.tile(np.array([cost]).transpose(), (1, 4))
margins = np.array(prices)-cost2

Secondary_dict={           # Propongo i prodotti più simili a quello mostrato --> problemino: 2 viene mostrato quasi sempre
    "Calabazas": [1,2],
    "Hinojo": [0,2],
    "Sesamo": [1,3],
    "Girasol": [2,4],
    "Amapola": [2,3]
}

res_price_params = {
    "shape": 20,
    "scale": 2,
    "max" : 60,
    "min" : 5
}

np.random.seed(1)
probabilities = np.random.uniform(0.0, 1, (5, 5))


alphas=[1/5, 1/5, 1/5, 1/5, 1/5] 
# per ora li generiamo così, tutti uguali -> devo generare 3 diversi vettori alpha

poisson_lambda = 1
#=valore atteso del numero di prodotti acquistati (specifico per prodotto)...non dipende dal
#prodotto oltre che dallo user che dal tipo di user che

p_users = [4/9, 3/9, 2/9] #probabilità di essere un tipo di utente-> da cambiare

lambda_q = 0.5 #just my idea of lambda
#possiamo stimarlo con i dati passati provenienti dal sito -> vino tot è stato comprato 15 volte

#proviamo a pensare, ha senso vederlo come coppia? categoria-prodotto? Avrei 3 categorie *5 prodotti-> 15 lambda diversi
# ^^ Andre: secondo me ha senso avere 3 lambda_q diversi per categoria MA non per prodotto. La probabilità di continuare
#           a guardare secondo me dipende dall'utente e non dal prodotto

for i in range (5):
    products.append(Product(prices[i], i, nameofproduct[i],margins[i]))

for i in range(3):
    users.append(UserCat(alphas, res_price_params, poisson_lambda, probabilities))

Env = Environment(users, products,  lambda_q, Secondary_dict, p_users)


In [27]:
probabilities

array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
        1.46755891e-01],
       [9.23385948e-02, 1.86260211e-01, 3.45560727e-01, 3.96767474e-01,
        5.38816734e-01],
       [4.19194514e-01, 6.85219500e-01, 2.04452250e-01, 8.78117436e-01,
        2.73875932e-02],
       [6.70467510e-01, 4.17304802e-01, 5.58689828e-01, 1.40386939e-01,
        1.98101489e-01],
       [8.00744569e-01, 9.68261576e-01, 3.13424178e-01, 6.92322616e-01,
        8.76389152e-01]])

In [28]:
Env.optimal_reward()

(4.930615861298395, [3, 3, 1, 0, 0])

In [29]:
Env.aggregated_reward([3, 3, 1, 0, 0])

4.930615861298395

In [30]:
Env.single_reward(0, [3, 3, 1, 0, 0])

4.930615861298395

In [6]:
Env.exp_return(products[4], [], [0], [Product([], -1, "null", [])], [0,0,0,0,0], users[2])

0.2355647769595931

In [4]:
[0 for x in Env.products]

[0, 0, 0, 0, 0]

In [5]:
a = [0,0,0,0,0]
b = a
b[3] +=1
a

[0, 0, 0, 1, 0]

In [7]:
np.argmax(a)

3

In [13]:
a = [0, 0, 0, 0, 0]
history = []
for i in range(10):
    j = i%5
    a[j]+= 1
    history.append(a.copy())
history

[[1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 1, 1, 0, 0],
 [1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 2, 1, 1, 1],
 [2, 2, 2, 1, 1],
 [2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2]]

In [14]:
a.copy()[3]

2